In [1]:
from __future__ import annotations

import re
from collections import Counter, defaultdict, deque
from dataclasses import dataclass
from enum import Enum
from functools import cache
from heapq import heappop, heappush
from itertools import combinations, cycle, product, zip_longest
from math import inf
from typing import *

import black
import jupyter_black
from parse import parse
from primefac import primefac

jupyter_black.load(lab=True, target_version=black.TargetVersion.PY310)


def first(iterable):
    return next(iter(iterable))

In [384]:
# Day 1: Chronal Calibration
def first_repeat(frequencies):
    freqs = cycle(frequencies)
    found = set()
    current = 0
    while current not in found:
        found.add(current)
        current += next(freqs)
    return current


lines = open("2018/1.txt").read().splitlines()
frequencies = [int(line) for line in lines]
print(f"Part 1: {sum(frequencies)}")  # 411
print(f"Part 2: {first_repeat(frequencies)}")  # 56360

Part 1: 411
Part 2: 56360


In [3]:
# Day 2: Inventory Management System
def checksum(ids):
    def contains(text, repeated):
        "`text` contains exactly `repeated` repeated characters"
        return repeated in Counter(text).values()

    twos = sum(contains(id, 2) for id in ids)
    threes = sum(contains(id, 3) for id in ids)
    return twos * threes


def common_letters(ids):
    "Common letters for the two ids that only differ by one character."
    for id_a, id_b in combinations(ids, 2):
        common = [ltr_a for ltr_a, ltr_b in zip(id_a, id_b) if ltr_a == ltr_b]
        if len(id_a) - len(common) == 1:
            return "".join(common)


ids = open("2018/2.txt").read().splitlines()
print(f"Part 1: {checksum(ids)}")  # 5166
print(f"Part 2: {common_letters(ids)}")  # cypueihajytordkgzxfqplbwn

Part 1: 5166
Part 2: cypueihajytordkgzxfqplbwn


In [6]:
# Day 3: No Matter How You Slice It
def patch(row, column, width, height):
    "Return a set with all coordinates"
    return {
        (r, c) for c in range(column, column + width) for r in range(row, row + height)
    }


def point_claims(claims):
    "Number of claims per point"
    patches = Counter()
    for _, column, row, width, height in claims:
        patches.update(patch(row, column, width, height))
    return patches


def overlapping_inches(claims):
    return sum(p > 1 for p in point_claims(claims).values())


def non_overlapping_claim_id(claims):
    overlapping_points = {
        coord for coord, overlaps in point_claims(claims).items() if overlaps > 1
    }

    for id, column, row, width, height in claims:
        claim = patch(row, column, width, height)
        if all(point not in overlapping_points for point in claim):
            return id


# ID, column, row, width, height
lines = open("2018/3.txt").read().splitlines()
claims = [tuple(int(x) for x in re.findall(r"\d+", line)) for line in lines]
print(f"Part 1: {overlapping_inches(claims)}")  # 115348
print(f"Part 2: {non_overlapping_claim_id(claims)}")  # 188

Part 1: 115348
Part 2: 188


In [15]:
# Day 4: Repose Record
def sleep_schedule(lines: List[str]) -> DefaultDict[int, Counter]:
    """
    Return a sleep schedule for all guards.

    Key is guard ID and value is a Counter with minute after midnight as key and number of
    times the guard sleeps during that minute as value.
    """
    log: List[List[str]] = sorted([line[1:].split("] ") for line in lines])
    guard = snooze = 0
    schedule = defaultdict(Counter)
    for timestamp, entry in log:
        if p := parse("Guard #{:d} begins shift", entry):
            guard = p[0]
        elif p := parse("falls asleep", entry):
            snooze = int(timestamp[-2:])
        elif p := parse("wakes up", entry):
            wakes = int(timestamp[-2:])
            assert wakes > snooze
            schedule[guard].update(range(snooze, wakes))
        else:
            raise ValueError("Can't parse", entry)
    return schedule


def part1(schedule: DefaultDict[int, Counter]) -> int:
    """
    Find the guard that has the most minutes asleep. What minute does that guard spend
    asleep the most?
    """
    guard, _ = sorted(
        [(guard, slept.total()) for guard, slept in schedule.items()],
        key=lambda x: x[1],
        reverse=True,
    )[0]
    minute = schedule[guard].most_common(1)[0][0]

    return guard * minute


def part2(schedule: DefaultDict[int, Counter]) -> int:
    """
    Of all guards, which guard is most frequently asleep on the same minute?
    """
    guard, (minute, _) = sorted(
        [(guard, slept.most_common(1)[0]) for guard, slept in schedule.items()],
        key=lambda x: x[1][1],
        reverse=True,
    )[0]

    return guard * minute


lines: str = open("2018/4.txt").read().splitlines()
schedule: DefaultDict[int, Counter] = sleep_schedule(lines)
print(f"Part 1: {part1(schedule)}")  # 35184
print(f"Part 2: {part2(schedule)}")  # 37886

Part 1: 35184
Part 2: 37886


In [270]:
# Day 5: Alchemical Reduction
def one_pass(polymers):
    result = []
    chars = zip_longest(polymers, polymers[1:], fillvalue=polymers[-1])
    for cur, nxt in chars:
        if cur == nxt.swapcase():
            next(chars)
        else:
            result.append(cur)
    return result


def reduce_all(text):
    polymers = list(text)
    length = 0
    while length != len(polymers):
        length = len(polymers)
        polymers = one_pass(polymers)
    return len(polymers)


def remove_char(char, text):
    table = {ord(char.lower()): None, ord(char.upper()): None}
    return text.translate(table)


def shortest_polymer(text):
    units = set(text.lower())
    lengths = {unit: reduce_all((remove_char(unit, text))) for unit in units}
    return min(lengths.values())


text = open("2018/5.txt").read().strip()
print(f"Part 1: {reduce_all(text)}")  # 9704, 3.5s runtime
print(f"Part 2: {shortest_polymer(text)}")  # 6942, 1m 10s

# Only use one pass, see
# https://www.reddit.com/r/adventofcode/comments/a3912m/comment/eb4ff0j/?utm_source=reddit&utm_medium=web2x&context=3 
def reduce_all(text):
    buf = []
    for c in text:
        if buf and c == buf[-1].swapcase():
            buf.pop()
        else:
            buf.append(c)
    return len(buf)


# shortest_polymer(text) # Finishes in 0.3s.

Part 1: 9704
Part 2: 6942


In [3]:
# Day 6: Chronal Coordinates
def min_coords(coords):
    return min(coord[0] for coord in coords), min(coord[1] for coord in coords)


def max_coords(coords):
    return max(coord[0] for coord in coords), max(coord[1] for coord in coords)


def points(coords):
    min_x, min_y = min_coords(coords)
    max_x, max_y = max_coords(coords)
    return {(x, y) for x in range(min_x, max_x + 1) for y in range(min_y, max_y + 1)}


def manhattan_distance(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])


@cache
def closest_coord(point, coords):
    "Return closest coordinate for a point, None if there is a tie."
    distances = {(coord): manhattan_distance(point, coord) for coord in coords}
    shortest_distance = min(distances.values())
    closest = [
        coord for coord, distance in distances.items() if distance == shortest_distance
    ]
    if len(closest) == 1:
        return first(closest)


def safe_coords(coords):
    def bounding_box(coords):
        min_x, min_y = min_coords(coords)
        max_x, max_y = max_coords(coords)
        top_bottom = {(y, x) for x in range(min_x, max_x + 1) for y in (min_y, max_y)}
        left_right = {(y, x) for y in range(min_y, max_y + 1) for x in (min_x, max_x)}
        return top_bottom | left_right

    # Infinite coords are the ones that are closest to any point in the bounding box
    infinite = {
        closest_coord(point, coords)
        for point in bounding_box(coords)
        if closest_coord(point, coords)
    }
    return set(coords) - infinite


def largest_area(coords):
    safe = safe_coords(coords)
    areas = Counter(
        closest_coord(point, coords)
        for point in points(coords)
        if closest_coord(point, coords) in safe
    )
    return first(areas.most_common(1))[1]


def distance_lt(coords, value=10_000):
    def distance_to_all(point, coords):
        return sum(manhattan_distance(point, coord) for coord in coords)

    return sum(distance_to_all(point, coords) < value for point in points(coords))


data = open("2018/6.txt").read().splitlines()
coords = tuple(tuple(int(x) for x in line.split(", ")) for line in data)
print("Part 1:", largest_area(coords))  # 2906
print("Part 2:", distance_lt(coords))  # 50530

Part 1: 2906
Part 2: 50530


In [14]:
# Day 7: The Sum of Its Parts
def starting_steps():
    children = {x for value in steps.values() for x in value}
    parents = {x for x in steps.keys()}
    return list(parents - children)


def path():
    neighbors = starting_steps()
    path = []

    while neighbors:
        neighbors.sort(reverse=True)
        current = neighbors.pop()
        path.append(current)
        neighbors.extend(
            child
            for child in steps[current]
            if all(parent in path for parent in needs[child])
        )

    return "".join(path)


@dataclass
class Worker:
    step: str = ""
    time_remaining: int = 0

    def add_step(self, step: str, offset=61):
        new_time = ord(step) - ord("A") + offset
        self.time_remaining = new_time
        self.step = step

    def tick(self):
        self.time_remaining -= 1

    def available(self):
        return self.time_remaining <= 0

    def finished_step(self):
        return self.time_remaining == 0


def complete():
    possible_steps = starting_steps()
    done = set()
    workers = [Worker() for _ in range(5)]

    for second in range(10**4):
        # Check who is done and add new possible neighbors
        for worker in workers:
            worker.tick()
            if worker.finished_step():
                done.add(worker.step)
                possible_steps.extend(
                    child
                    for child in steps[worker.step]
                    if all(parent in done for parent in needs[child])
                )
        possible_steps.sort(reverse=True)
        # Assign all possible steps to workers
        for worker in workers:
            if worker.available() and possible_steps:
                worker.add_step(possible_steps.pop())
        # Check if we are done
        if all(w.available() for w in workers):
            return second


lines = open("2018/7.txt").read().splitlines()
steps: DefaultDict[str, List[str]] = defaultdict(list)
needs: DefaultDict[str, List[str]] = defaultdict(list)

for line in lines:
    parent, child = parse("Step {} must be finished before step {} can begin.", line)
    steps[parent].append(child)
    needs[child].append(parent)

print(f"Part 1: {path()}")  # EFHLMTKQBWAPGIVXSZJRDUYONC
print(f"Part 2: {complete()}")  # 1056

Part 1: EFHLMTKQBWAPGIVXSZJRDUYONC
Part 2: 1056


In [17]:
# Day 8: Memory Maneuver
@dataclass
class Node:
    children: List[Node]
    metadata: List[int]


def create_tree(numbers):
    child_quantity = next(numbers)
    metadata_quantity = next(numbers)
    return Node(
        [create_tree(numbers) for _ in range(child_quantity)],
        [next(numbers) for _ in range(metadata_quantity)],
    )


def sum_metadata(node):
    return sum(node.metadata) + sum(sum_metadata(child) for child in node.children)


def value(node):
    if not node.children:
        return sum(node.metadata)
    return sum(
        value(node.children[index - 1])
        for index in node.metadata
        if index > 0 and index <= len(node.children)
    )


line = open("2018/8.txt").read().strip()
numbers = iter(int(x) for x in line.split())
tree = create_tree(numbers)
print(f"Part 1: {sum_metadata(tree)}")  # 49180
print(f"Part 2: {value(tree)}")  # 20611

Part 1: 49180
Part 2: 20611


In [18]:
# Day 9: Marble Mania
def play_rounds(num_players, last_marble):
    scores = defaultdict(int)
    marbles = deque()
    for marble in range(last_marble + 1):
        if marble % 23 == 0 and marble:
            player = 1 + (marble - 1) % num_players
            marbles.rotate(7)
            scores[player] += marbles.pop() + marble
            marbles.rotate(-1)
        else:
            marbles.rotate(-1)
            marbles.append(marble)
    return max(scores.values())


# My input: 477 players; last marble is worth 70851 points
print(f"Part 1: {play_rounds(477, 70851)}")  # 374690
print(f"Part 2: {play_rounds(477, 70851*100)}")  # 3009951158

Part 1: 374690
Part 2: 3009951158


In [5]:
# Day 10: The Stars Align
def tick(points, seconds=1):
    def tick_one(point):
        point[0] += seconds * point[2]  # X + X velocity
        point[1] += seconds * point[3]  # Y + Y velocity
        return point

    return [tick_one(point) for point in points]


def message_appears(points):
    "Return the time when height reaches a minimum"
    min_spread = 10**6
    for second in range(10**6):
        if (spread := max(points)[0] - min(points)[0]) < min_spread:
            min_spread = spread
        else:
            # We have moved past minimum, step back once
            points = tick(points, -1)
            return second - 1
        points = tick(points)


def print_message(points):
    min_x = min(x for x, y, _, _ in points)
    max_x = max(x for x, y, _, _ in points)
    min_y = min(y for x, y, _, _ in points)
    max_y = max(y for x, y, _, _ in points)
    filled = {(x, y) for (x, y, _, _) in points}
    for y in range(min_y, max_y + 1):
        print("".join("⬜️🟦"[((x, y) in filled) * 2] for x in range(min_x, max_x + 1)))


lines = open("2018/10.txt").read().splitlines()
# x, y, x velocity, y velocity
points = [[int(x) for x in re.findall(r"-?\d+", line)] for line in lines]
print(
    "Part 2:", message_appears(points)
)  # 10619 - which happens to be the answer to part 2(!)
print_message(points)  # ABGXJBXF

Part 2: 10619
⬜⬜🟦🟦⬜⬜⬜⬜🟦🟦🟦🟦🟦⬜⬜⬜⬜🟦🟦🟦🟦⬜⬜⬜🟦⬜⬜⬜⬜🟦⬜⬜⬜⬜⬜🟦🟦🟦⬜⬜🟦🟦🟦🟦🟦⬜⬜⬜🟦⬜⬜⬜⬜🟦⬜⬜🟦🟦🟦🟦🟦🟦
⬜🟦⬜⬜🟦⬜⬜⬜🟦⬜⬜⬜⬜🟦⬜⬜🟦⬜⬜⬜⬜🟦⬜⬜🟦⬜⬜⬜⬜🟦⬜⬜⬜⬜⬜⬜🟦⬜⬜⬜🟦⬜⬜⬜⬜🟦⬜⬜🟦⬜⬜⬜⬜🟦⬜⬜🟦⬜⬜⬜⬜⬜
🟦⬜⬜⬜⬜🟦⬜⬜🟦⬜⬜⬜⬜🟦⬜⬜🟦⬜⬜⬜⬜⬜⬜⬜⬜🟦⬜⬜🟦⬜⬜⬜⬜⬜⬜⬜🟦⬜⬜⬜🟦⬜⬜⬜⬜🟦⬜⬜⬜🟦⬜⬜🟦⬜⬜⬜🟦⬜⬜⬜⬜⬜
🟦⬜⬜⬜⬜🟦⬜⬜🟦⬜⬜⬜⬜🟦⬜⬜🟦⬜⬜⬜⬜⬜⬜⬜⬜🟦⬜⬜🟦⬜⬜⬜⬜⬜⬜⬜🟦⬜⬜⬜🟦⬜⬜⬜⬜🟦⬜⬜⬜🟦⬜⬜🟦⬜⬜⬜🟦⬜⬜⬜⬜⬜
🟦⬜⬜⬜⬜🟦⬜⬜🟦🟦🟦🟦🟦⬜⬜⬜🟦⬜⬜⬜⬜⬜⬜⬜⬜⬜🟦🟦⬜⬜⬜⬜⬜⬜⬜⬜🟦⬜⬜⬜🟦🟦🟦🟦🟦⬜⬜⬜⬜⬜🟦🟦⬜⬜⬜⬜🟦🟦🟦🟦🟦⬜
🟦🟦🟦🟦🟦🟦⬜⬜🟦⬜⬜⬜⬜🟦⬜⬜🟦⬜⬜🟦🟦🟦⬜⬜⬜⬜🟦🟦⬜⬜⬜⬜⬜⬜⬜⬜🟦⬜⬜⬜🟦⬜⬜⬜⬜🟦⬜⬜⬜⬜🟦🟦⬜⬜⬜⬜🟦⬜⬜⬜⬜⬜
🟦⬜⬜⬜⬜🟦⬜⬜🟦⬜⬜⬜⬜🟦⬜⬜🟦⬜⬜⬜⬜🟦⬜⬜⬜🟦⬜⬜🟦⬜⬜⬜⬜⬜⬜⬜🟦⬜⬜⬜🟦⬜⬜⬜⬜🟦⬜⬜⬜🟦⬜⬜🟦⬜⬜⬜🟦⬜⬜⬜⬜⬜
🟦⬜⬜⬜⬜🟦⬜⬜🟦⬜⬜⬜⬜🟦⬜⬜🟦⬜⬜⬜⬜🟦⬜⬜⬜🟦⬜⬜🟦⬜⬜⬜🟦⬜⬜⬜🟦⬜⬜⬜🟦⬜⬜⬜⬜🟦⬜⬜⬜🟦⬜⬜🟦⬜⬜⬜🟦⬜⬜⬜⬜⬜
🟦⬜⬜⬜⬜🟦⬜⬜🟦⬜⬜⬜⬜🟦⬜⬜🟦⬜⬜⬜🟦🟦⬜⬜🟦⬜⬜⬜⬜🟦⬜⬜🟦⬜⬜⬜🟦⬜⬜⬜🟦⬜⬜⬜⬜🟦⬜⬜🟦⬜⬜⬜⬜🟦⬜⬜🟦⬜⬜⬜⬜⬜
🟦⬜⬜⬜⬜🟦⬜⬜🟦🟦🟦🟦🟦⬜⬜⬜⬜🟦🟦🟦⬜🟦⬜⬜🟦⬜⬜⬜⬜🟦⬜⬜⬜🟦🟦🟦⬜⬜⬜⬜🟦🟦🟦🟦🟦⬜⬜⬜🟦⬜⬜⬜⬜🟦⬜⬜🟦⬜⬜⬜⬜⬜


In [7]:
# Day 11: Chronal Charge
# Using a summed-area table: https://en.wikipedia.org/wiki/Summed-area_table
@cache
def power_level(x, y, serial):
    def hundreds_digit(num):
        return (num // 100) % 10

    rack_id = x + 10
    level = rack_id * y + serial
    level = hundreds_digit(level * rack_id) - 5
    return level


@cache
def summed_area_table(serial):
    I = defaultdict(int)
    for x in range(1, 301):
        for y in range(1, 301):
            I[(x, y)] = (
                power_level(x, y, serial)
                + I[(x, y - 1)]
                + I[(x - 1, y)]
                - I[(x - 1, y - 1)]
            )
    return I


def table_power_level(x, y, size, I):
    "Calculate power level for a square with x and y in top left based on summed area table I."
    return (
        I[(x + size - 1, y + size - 1)]
        + I[(x - 1, y - 1)]
        - I[(x + size - 1, y - 1)]
        - I[(x - 1, y + size - 1)]
    )


def max_coords(serial=1788, size=3):
    return max(
        (table_power_level(x, y, size, I=summed_area_table(serial)), (x, y))
        for x in range(1, 301 - size)
        for y in range(1, 301 - size)
    )


def max_size(serial=1788):
    return max((max_coords(serial, size), size) for size in range(1, 300))


print("Part 1:", max_coords())  # 235, 35
print("Part 2:", max_size())  # 142, 265, 7

Part 1: (31, (235, 35))
Part 2: ((65, (142, 265)), 7)


In [118]:
# Day 12: Subterranean Sustainability
def key(state, i):
    return (
        state.get(i - 2, ".")
        + state.get(i - 1, ".")
        + state.get(i, ".")
        + state.get(i + 1, ".")
        + state.get(i + 2, ".")
    )


def generate(state):
    lower = min(state.keys()) - 2
    upper = max(state.keys()) + 3
    new_state = {}
    for i in range(lower, upper):
        if rules.get(key(state, i)) == "#":
            new_state[i] = "#"
    return new_state


lines = open("2018/12.txt").read().splitlines()
state_str = lines[0].split()[-1]
rules = {line.split(" => ")[0]: line.split(" => ")[1] for line in lines[2:]}
state = {pos: char for pos, char in enumerate(state_str)}

for i in range(20):
    state = generate(state)
pot_sum = sum(key for key, value in state.items() if value == "#")
print("Part 1:", pot_sum)  # 1184

# Use this code to show that the pattern converges
# for i in range(20, 150):
#     state = generate(state)
#     pot_sum = sum(key for key, value in state.items() if value == "#")
#     pots = list(state.keys())
#     print(f"{i+1:4}: ({pot_sum}) {pots}")

# After 101 iterations the pattern stabilises and pot value increases with 5 for each
# step. The pot_value after 101 iterations is 724
def pot_value(x):
    return 724 + (x - 101) * 5


pot_sum = sum(key for key, value in state.items() if value == "#")
print("Part 2:", pot_value(50_000_000_000))  # 250000000219

Part 1: 1184
Part 2: 250000000219


In [19]:
# Day 13: Mine Cart Madness
@dataclass
class Cart:
    x: int
    y: int
    dir: int  # 0123 = NESW = '^>v<'
    next_turn: int = 0  # 012 left, straight, right

    def right(self):
        self.dir = (self.dir + 1) % 4

    def left(self):
        self.dir = (self.dir - 1) % 4

    def update_turn(self):
        self.next_turn = (self.next_turn + 1) % 3

    def tick(self, grid):
        DIRS = [(0, -1), (1, 0), (0, 1), (-1, 0)]  # NESW

        # Move forward (should always be possible)
        self.x += DIRS[self.dir][0]
        self.y += DIRS[self.dir][1]

        # Turn
        match grid[self.y][self.x], self.next_turn, self.dir:
            case "+", 0, _:
                self.left()
                self.update_turn()
            case "+", 1, _:
                self.update_turn()
            case "+", 2, _:
                self.right()
                self.update_turn()
            case "/", _, (0 | 2):
                self.right()
            case "/", _, (1 | 3):
                self.left()
            case "\\", _, (0 | 2):
                self.left()
            case "\\", _, (1 | 3):
                self.right()


def carts_and_grid(lines):
    # Create carts and replace them with lines in grid
    grid = [list(line) for line in lines]
    cart_symbols = "^>v<"
    carts = []
    for y, x in product(range(len(grid)), range(len(grid[0]))):
        if grid[y][x] in cart_symbols:
            dir = cart_symbols.find(grid[y][x])
            carts.append(Cart(x, y, dir))
            # Replace cart with rail in grid
            if dir in [0, 2]:
                grid[y][x] = "|"
            elif dir in [1, 3]:
                grid[y][x] = "-"
            else:
                raise ValueError(dir)
    return carts, grid


def tick_all(carts, grid):
    for cart in sorted(carts, key=lambda c: (c.y, c.x)):
        cart.tick(grid)


def crash_site(carts):
    coords = set()
    for cart in carts:
        if (cart.x, cart.y) in coords:
            return (cart.x, cart.y)
        coords.add((cart.x, cart.y))
    return False


def remove_crashed(carts, grid):
    while len(carts) > 1:
        crashed = []
        for cart in sorted(carts, key=lambda c: (c.y, c.x)):
            cart.tick(grid)
            if crash_site(carts):
                crashed = [
                    cart for cart in carts if (cart.x, cart.y) == crash_site(carts)
                ]
        carts = [cart for cart in carts if cart not in crashed]
    return carts[0].x, carts[0].y


lines = open("2018/13.txt").read().splitlines()
carts, grid = carts_and_grid(lines)

while not crash_site(carts):
    tick_all(carts, grid)
print("Part 1:", crash_site(carts))  # 86,118
print("Part 2:", remove_crashed(carts, grid))  # 2,81

Part 1: (86, 118)
Part 2: (2, 81)


In [24]:
# Day 14: Chocolate Charts
@cache  # 55s -> 43s execution time
def new_recipes(recipe_one, recipe_two):
    return [int(x) for x in str(recipe_one + recipe_two)]


def one_round(first, second, recipes):
    def new_position(current):
        return (current + recipes[current] + 1) % len(recipes)

    recipes.extend(new_recipes(recipes[first], recipes[second]))
    return new_position(first), new_position(second)


def initial_setup(start="37"):
    recipes = [int(x) for x in start]
    return 0, 1, recipes


def scores(rounds):
    "Return last 10 scores after `rounds`"
    first, second, recipes = initial_setup()
    for _ in range(rounds + 10):
        first, second = one_round(first, second, recipes)
    return to_string(recipes[rounds : rounds + 10])


def to_string(numbers):
    return "".join(str(x) for x in numbers)


def first_sequence(sequence):
    first, second, recipes = initial_setup()
    for _ in range(10**9):
        first, second = one_round(first, second, recipes)
        if sequence in to_string(recipes[-7:]):
            return len(recipes) - 7 + to_string(recipes[-7:]).find(sequence)


print(f"Part 1: {scores(110201)}")  # 6107101544
print(f"Part 2: {first_sequence('110201')}")  # 20291131 (takes about 1 minute)

Part 1: 6107101544
Part 2: 20291131


In [25]:
# Day 14: https://www.reddit.com/r/adventofcode/comments/a61ojp/comment/ebr8abv/?utm_source=reddit&utm_medium=web2x&context=3
recipes = "110201"

score = "37"
elf1 = 0
elf2 = 1
while recipes not in score[-7:]:
    score += str(int(score[elf1]) + int(score[elf2]))
    elf1 = (elf1 + int(score[elf1]) + 1) % len(score)
    elf2 = (elf2 + int(score[elf2]) + 1) % len(score)

print("Part 1:", score[int(recipes) : int(recipes) + 10])
print("Part 2:", score.index(recipes))

Part 1: 6107101544
Part 2: 20291131


In [114]:
# Day 15: Beverage Bandits
def print_cave():
    for row, line in enumerate(cave):
        for column, _ in enumerate(line):
            print(cave[row][column], end="")
        print()
    print()


def calculate_costs(start):
    """
    Cost to move from position 'start' to any other position in cave
    """
    frontier = []
    heappush(frontier, (0, start))
    came_from = {}
    cost_so_far = defaultdict(lambda: inf)
    came_from[start] = None
    cost_so_far[start] = 0

    while frontier:
        current = heappop(frontier)[1]
        for next in valid_moves(current):
            new_cost = cost_so_far[current] + 1
            if new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost
                heappush(frontier, (priority, next))
                came_from[next] = current
    return came_from, cost_so_far


def valid_moves(current):
    candidates = []
    (row, column) = current
    # The cave has a border all around, no need to check for boundaries
    if cave[row][column + 1] == ".":
        candidates.append((row, column + 1))
    if cave[row + 1][column] == ".":
        candidates.append((row + 1, column))
    if cave[row - 1][column] == ".":
        candidates.append((row - 1, column))
    if cave[row][column - 1] == ".":
        candidates.append((row, column - 1))
    return candidates


def target_in_range(player):
    (row, column) = player
    neighbors = (
        cave[row][column + 1]
        + cave[row + 1][column]
        + cave[row - 1][column]
        + cave[row][column - 1]
    )
    return enemy(player) in neighbors


def value(position):
    (row, column) = position
    return cave[row][column]


def enemy(position):
    "Return enemy race for player in `position`."
    return "G" if value(position) == "E" else "E"


def move_and_attack(player, elf_power):
    "Return enemy type that died after attack, . if no death"
    def move_to(from_, to_):
        "Helper function to move both character and hit points"
        (row, column) = from_
        (to_row, to_column) = to_
        cave[to_row][to_column] = value(from_)
        hit_points[to_row][to_column] = hit_points[row][column]
        cave[row][column] = "."
        hit_points[row][column] = 0

    if value(player) == ".":
        # Player is dead, don't continue with round
        return
    if not target_in_range(player):
        # Move
        came_from, cost = calculate_costs(player)
        reachable = {
            pos: cost[pos]
            for r, line in enumerate(cave)
            for c, candidate in enumerate(line)
            if candidate == enemy(player)
            for pos in valid_moves((r, c))
            if pos in cost.keys()
        }
        nearest = sorted(
            pos for pos in reachable.keys() if reachable[pos] == min(reachable.values())
        )
        if nearest:
            chosen = first(nearest)
            # Backtrack to find first step to take
            pos = chosen
            while pos != player:
                first_step = pos
                pos = came_from[pos]
            move_to(player, first_step)
            player = first_step
    return attack(player, elf_power)


def attack(player, elf_power):
    "Attack enemy. Return enemy type that died, . if no death"
    (row, column) = player
    candidates = [
        (row - 1, column),
        (row, column - 1),
        (row, column + 1),
        (row + 1, column),
    ]
    enemies = {
        (r, c): hit_points[r][c] for (r, c) in candidates if cave[r][c] == enemy(player)
    }
    if enemies:
        # We have someone to attack. Pick out enemy with lowest HP and first in order
        (row_hit, column_hit) = first(
            sorted(pos for pos, hp in enemies.items() if hp == min(enemies.values()))
        )
        power = 3  # Goblin power is always 3
        if value(player) == "E":
            power = elf_power
        hit_points[row_hit][column_hit] -= power
        if hit_points[row_hit][column_hit] <= 0:
            # Enemy killed
            dies = cave[row_hit][column_hit]
            cave[row_hit][column_hit] = "."
            return dies
    return "."


def score():
    return sum(point for line in hit_points for point in line if point > 0)


def occurences(char):
    "Count number of occurences of `char`."
    return sum(1 for row in cave for x in row if x in char)


def play_game(elf_power=3):
    "Play a full game. Return number of killed elves and number of rounds."
    def init_map():
        global cave, hit_points
        lines = open("2018/15.txt").read().splitlines()
        cave = [[val for val in line] for line in lines]
        hit_points = [[200 if val in "EG" else 0 for val in line] for line in lines]

    def one_round(elf_power):
        "Try to move all players. Return true if game over."
        game_over = False
        players_to_move = [
            (row, column)
            for row, line in enumerate(cave)
            for column, player in enumerate(line)
            if cave[row][column] in "GE"
        ]
        for player in players_to_move:
            move_and_attack(player, elf_power)
        if not occurences("E") or not occurences("G"):
            game_over = True
        return game_over

    init_map()
    elves = occurences("E")
    rounds = 0
    while not one_round(elf_power):
        rounds += 1
    return abs(occurences("E") - elves), rounds


dead_elves, rounds = play_game()
print(f"Part 1: {rounds * score()}")  # 217890

elf_power = 4
while dead_elves > 0:
    dead_elves, rounds = play_game(elf_power)
    elf_power += 1

print(f"Part 2: {rounds * score()}")  # 43645

Part 1: 217890
Part 2: 43645


In [103]:
# Day 16: Chronal Classification
def addr(registers, a, b, c):
    result = registers[:]
    result[c] = result[a] + result[b]
    return result


def addi(registers, a, b, c):
    result = registers[:]
    result[c] = result[a] + b
    return result


def mulr(registers, a, b, c):
    result = registers[:]
    result[c] = result[a] * result[b]
    return result


def muli(registers, a, b, c):
    result = registers[:]
    result[c] = result[a] * b
    return result


def banr(registers, a, b, c):
    result = registers[:]
    result[c] = result[a] & result[b]
    return result


def bani(registers, a, b, c):
    result = registers[:]
    result[c] = result[a] & b
    return result


def borr(registers, a, b, c):
    result = registers[:]
    result[c] = result[a] | result[b]
    return result


def bori(registers, a, b, c):
    result = registers[:]
    result[c] = result[a] | b
    return result


def setr(registers, a, b, c):
    result = registers[:]
    result[c] = result[a]
    return result


def seti(registers, a, b, c):
    result = registers[:]
    result[c] = a
    return result


def gtir(registers, a, b, c):
    result = registers[:]
    result[c] = int(a > result[b])
    return result


def gtri(registers, a, b, c):
    result = registers[:]
    result[c] = int(result[a] > b)
    return result


def gtrr(registers, a, b, c):
    result = registers[:]
    result[c] = int(result[a] > result[b])
    return result


def eqir(registers, a, b, c):
    result = registers[:]
    result[c] = int(a == result[b])
    return result


def eqri(registers, a, b, c):
    result = registers[:]
    result[c] = int(result[a] == b)
    return result


def eqrr(registers, a, b, c):
    result = registers[:]
    result[c] = int(result[a] == result[b])
    return result


def matches(before, instruction, after):
    # fmt: off
    opcodes = [
        addr, addi,
        mulr, muli,
        banr, bani,
        borr, bori,
        setr, seti,
        gtir, gtri, gtrr,
        eqir, eqri, eqrr 
    ]
    # fmt: on
    matches = set()
    for opcode in opcodes:
        if opcode(before, *instruction[1:]) == after:
            matches.add(opcode)
    return matches


def parse_instruction(sample):
    chunk = [int(x) for x in re.findall(r"\d+", sample)]
    before = chunk[0:4]
    instruction = chunk[4:8]
    after = chunk[8:12]
    assert len(before) == len(instruction) == len(after) == 4
    return before, instruction, after


def three_matches_or_more(samples):
    three_plus_matches = 0
    for sample in samples:
        before, instruction, after = parse_instruction(sample)
        if len(matches(before, instruction, after)) >= 3:
            three_plus_matches += 1
    return three_plus_matches


def disassemble_opcodes(samples):
    known_opcodes = defaultdict(int)
    for sample in samples:
        before, instruction, after = parse_instruction(sample)
        possible_opcodes = matches(before, instruction, after)
        unknown = possible_opcodes - {op for op in known_opcodes.values()}
        if len(unknown) == 1:
            known_opcodes[instruction[0]] = first(unknown)
    return known_opcodes


def execute_program(program, opcodes):
    registers = [0] * 4
    for line in program:
        registers = opcodes[line[0]](registers, *line[1:])
    return registers[0]


parts = open("2018/16.txt").read().split("\n\n\n\n")
samples = parts[0].split("\n\n")
print(f"Part 1: {three_matches_or_more(samples)}")  # 677
program = [
    [int(x) for x in re.findall(r"\d+", line)] for line in parts[1].strip().split("\n")
]
print(f"Part 2: {execute_program(program, disassemble_opcodes(samples))}")  # 540

Part 1: 677
Part 2: 540


In [575]:
# Day 17: Reservoir Research
def create_clay(lines):
    clay = set()
    for line in lines:
        if p := parse("x={:d}, y={:d}..{:d}\n", line):
            x, start, stop = p
            for y in range(start, stop + 1):
                clay.add((x, y))
        elif p := parse("y={:d}, x={:d}..{:d}\n", line):
            y, start, stop = p
            for x in range(start, stop + 1):
                clay.add((x, y))
        else:
            raise ValueError("Can't parse", line)
    return clay


def wall(position, direction):
    "Return x position if there is a clay wall somewhere in direction (-1 for left and +1 for right) and there is either clay or water underneath all positions up until that wall."
    (x, y) = position
    while (x, y + 1) in clay or (x, y + 1) in still_water:
        x = x + direction
        if (x, y) in clay:
            return x
    return False


def next_positions(position):
    "Flow water and return possible next positions"

    def pass_through(x, y, direction):
        "Flow water on one row until there is nothing below, return last point. Add points visited to `passed_through`."
        while (x, y + 1) in clay or (x, y + 1) in still_water:
            passed_through.add((x, y))
            x = x + direction
        return (x, y)

    (x, y) = position
    # Water flows down if possible
    candidate = (x, y + 1)
    if candidate not in clay and candidate not in still_water:
        passed_through.add(position)
        return {candidate}

    right = wall(position, 1)
    left = wall(position, -1)
    if right and left:
        # print(f"Fill row {y} with water")
        for x_ in range(left + 1, right):
            still_water.add((x_, y))
        return {(x, y - 1)}
    elif right:
        # Skip to the right, next to wall
        x = right - 1
        return {pass_through(x, y, -1)}
    elif left:
        # Skip to the left, next to wall
        x = left + 1
        return {pass_through(x, y, +1)}
    else:
        # print(f"Overflow both sides, {position}")
        # Move to the left until there is air below
        drops = {pass_through(x, y, -1)}
        # Move to the right until there is air below
        (x, y) = position
        drops.add(pass_through(x, y, +1))
        return drops


@cache
def y_max():
    return max(clay, key=lambda pt: pt[1])[1]


@cache
def y_min():
    return min(clay, key=lambda pt: pt[1])[1]


def flow(start):
    frontier = {start}
    while frontier:
        current = frontier.pop()
        for drop in next_positions(current):
            if drop[1] <= y_max() and drop not in still_water:
                frontier.add(drop)


def print_clay(from_x, to_x, from_y, to_y):
    for y in range(from_y, to_y):
        print(f"{y:4}", end="")
        for x in range(from_x, to_x):
            if (x, y) in clay:
                print("#", end="")
            elif (x, y) in still_water:
                print("~", end="")
            elif (x, y) in passed_through:
                print("|", end="")
            else:
                print(".", end="")
        print()


clay = create_clay(open("2018/17.txt").readlines())
still_water, passed_through = set(), set()

flow((500, y_min()))
print(f"Part 1: {len(still_water | passed_through)}")  # 31883
print(f"Part 2: {len(still_water)}")  # 24927

Part 1: 31883
Part 2: 24927


In [754]:
# Day 18: Settlers of The North Pole
def neighbors(area, point):
    (r, c) = point
    # fmt:off
    candidates = {(r - 1, c - 1), (r - 1, c), (r - 1, c + 1),
                  (r    , c - 1),             (r    , c + 1),
                  (r + 1, c - 1), (r + 1, c), (r + 1, c + 1)}
    # fmt:on
    return {p for p in candidates if p in area}


def side(area):
    return max(area)[0] + 1


def print_area(area):
    # Print a square area
    for row in range(0, side(area)):
        for column in range(0, side(area)):
            print(area[(row, column)], end="")
        print()
    print()


def adjacent_acre_count(area, point, char):
    "Number of adjacent acres containing `char`."
    return sum(1 for pt in neighbors(area, point) if area[pt] == char)


def tick(area):
    new_area = {}
    for point in product(range(side(area)), repeat=2):
        match area[point]:
            case ".":
                if adjacent_acre_count(area, point, "|") >= 3:
                    new_area[point] = "|"
                else:
                    new_area[point] = "."
            case "|":
                if adjacent_acre_count(area, point, "#") >= 3:
                    new_area[point] = "#"
                else:
                    new_area[point] = "|"
            case "#":
                if (
                    adjacent_acre_count(area, point, "#") >= 1
                    and adjacent_acre_count(area, point, "|") >= 1
                ):
                    new_area[point] = "#"
                else:
                    new_area[point] = "."
    return new_area


def resource_value(area):
    "Wooded acers * lumberyards"
    counts = Counter(area.values())
    return counts["|"] * counts["#"]


lines = open("2018/18.txt").readlines()
area = {
    (row, column): char
    for (row, line) in enumerate(lines)
    for (column, char) in enumerate(line.strip())
}

values = {}
for i in range(1, 700):
    area = tick(area)
    values[i] = resource_value(area)
print(f"Part 1: {values[10]}")  # 506385

# Resource values cycles through these values:
Counter(values.values())
# Looking at this we see that most resource values appears only once, but once we reach
# 567 generations the numbers start to repeat. 221676 appears twice and the cycle length
# is 28. Let's use generation 600 as the start of our calculated resource value.
cycles = list(values.values())[599 : 599 + 28]
# The resource value for a generation can then be found in the list cycles. Subtract 600
# and modulo 28 and use as index in the cycles list and you have the resource value for
# any generation > 600
generations = 1_000_000_000
print(f"Part 2: {cycles[(generations - 600) % 28]}")  # 215404

Part 1: 506385
Part 2: 215404


In [104]:
# Day 19: Go With The Flow
# Continuation of Day 16.
def execute_program(program, pc_reg):
    registers = [0] * 6
    pc = 0
    while pc < len(program):
        registers[pc_reg] = pc
        # print(f"ip={pc} {registers} {program[pc]} ", end="")
        registers = eval(program[pc])
        # print(f"{registers}")
        pc = registers[pc_reg] + 1
    return registers


def create_program(lines):
    program = []
    for line in lines[1:]:
        current = line.split()
        current.insert(1, "(registers, ")
        current.insert(3, ", ")
        current.insert(5, ", ")
        current.insert(7, ")")
        program.append("".join(current))
    return program


lines = open("2018/19.txt").read().splitlines()
pc_reg = first(int(x) for x in re.findall(r"\d", lines[0]))
program = create_program(lines)
registers = execute_program(program, pc_reg)  # 1m 20s
# register 0 is the sum of all divisors to 900 which in turn is loaded in register 1
print(f"Part 1: {registers[0]}")  # 2821


def execute_program_2(program, pc_reg):
    registers = [1] + [0] * 5
    pc = 0
    ticks = range(100)
    for _ in ticks:
        registers[pc_reg] = pc
        registers = eval(program[pc])
        pc = registers[pc_reg] + 1
    return registers


# Looking at this output we can see that reg 1 is loaded with 10551300. The program then
# goes on to add up all divisors to 10551300 which will take forever. Let's do it faster.


def divisor_sum(factors):
    """Return sum of all divisors given factors of a number.

    See
    https://www.amansmathsblogs.com/factors-formula-how-to-find-sum-of-factors-of-composite-numbers/#Short_Trick_To_Find_Sum_of_All_Factors_of_Composite_Numbers
    for formula
    """
    result = 1
    for factor in set(factors):
        base = factor
        power = factors.count(base) + 1
        result = result * (base**power - 1) / (base - 1)
    return int(result)


registers = execute_program_2(program, pc_reg)
print(f"Part 2: {divisor_sum(list(primefac(registers[1])))}")  # 30529296

Part 1: 2821
Part 2: 30529296


In [100]:
# Day 20: A Regular Map
def create_grid(expression):
    grid = {}
    min_cost_to_reach = defaultdict(lambda: inf)
    current = (0, 0)
    min_cost_to_reach[current] = 0
    positions = []  # Stack to keep tabs of positions at branching points
    for step in expression:
        (row, column) = current
        previous = current
        match step:
            case "N":
                grid[(row - 1, column)] = "-"
                row -= 2
                current = (row, column)
            case "S":
                grid[(row + 1, column)] = "-"
                row += 2
                current = (row, column)
            case "E":
                grid[((row, column + 1))] = "|"
                column += 2
                current = (row, column)
            case "W":
                grid[((row, column - 1))] = "|"
                column -= 2
                current = (row, column)
            case "(":
                positions.append(current)
            case ")":
                current = positions.pop()
            case "|":
                current = positions[-1]
        min_cost_to_reach[current] = min(
            (min_cost_to_reach[previous] + 1), (min_cost_to_reach[current])
        )
        grid[current] = "."
    return grid, min_cost_to_reach


def min_row(points):
    return min(points)[0]


def max_row(points):
    return max(points)[0]


def min_column(points):
    return min(points, key=lambda pt: pt[1])[1]


def max_column(points):
    return max(points, key=lambda pt: pt[1])[1]


def print_grid(grid, start=None):
    for row in range(min_row(grid) - 1, max_row(grid) + 2):
        for column in range(min_column(grid) - 1, max_column(grid) + 2):
            if (row, column) == start:
                print("X", end="")
            elif (row, column) in grid:
                print(grid[(row, column)], end="")
            else:
                print("#", end="")
        print()
    print()


expression = open("2018/20.txt").read().strip()
grid, min_costs = create_grid(expression)
# print_grid(grid, start=(0, 0))
print(f"Part 1: {max(min_costs.values())}")  # 4239
print(f"Part 2: {sum(1 for path in min_costs.values() if path >= 1000)}")

Part 1: 4239
Part 2: 8205


In [226]:
# Day 21: Chronal Conversion
def outer_loop(reg2, reg5):
    r2 = (reg2 + (reg5 & 0xFF)) * 65899 & 16777215  # row 12 and 13 in program
    r5 = reg5 // 256  # This is the inner loop
    return r2, r5


def outermost_loop(reg2, reg5):
    r5 = reg2 | 65536  # row 8 in program
    r2 = 4843319  # row 9 in program
    return r2, r5


def end_conditions():
    end_conditions = []
    r2, r5 = 4843319, 65536
    for _ in range(200000):
        r2, r5 = outer_loop(r2, r5)
        if r5 < 256:
            r2, _ = outer_loop(r2, r5)
            if r2 in end_conditions:
                return end_conditions
            else:
                end_conditions.append(r2)
            r2, r5 = outermost_loop(r2, r5)


# I'm guessing only 65899 and 4843319 are unique and all machines use the same bit length
values = end_conditions()
print(f"Part 1: {values[0]}")  # 8797248
print(f"Part 2: {values[-1]}")  # 3007673

Part 1: 8797248
Part 2: 3007673


Thoughts when trying to understand 21.txt
```
We are working with 24 bit registers

Outermost loop (pc = 6):
reg 5 = reg 2 | 65536 (i.e. set bit 16 of reg 2 to 1 and put in reg 5)
reg 2 = 4843319

        Outer loop (pc = 8):
        reg 2 = (reg 2 + (reg 5 & 0xFF)) * 65899 & 16777215
        if reg 4 < 256:
                goto final_check

        reg 5 = reg 5 // 256 (this is all inner loop does)
        Set reg 5 = reg 4
        Goto Outer loop

final_check:
end program if reg 2 == reg 0
goto outermost loop
```

In [68]:
# Day 22: Mode Maze
class Tool(Enum):
    NEITHER = 0
    TORCH = 1
    CLIMBING_GEAR = 2

    def __lt__(self, other):
        return self.value < other.value


class Region(Enum):
    ROCKY = 0
    WET = 1
    NARROW = 2


def geologic_index(x, y):
    if x == 0 or y == 0:
        return 16807 * x + 48271 * y
    elif x == x_target and y == y_target:
        return 0
    else:
        return erosion_level(x - 1, y) * erosion_level(x, y - 1)


@cache
def erosion_level(x, y):
    return (geologic_index(x, y) + depth) % 20183


@cache
def region_type(x, y):
    return Region(erosion_level(x, y) % 3)


def risk_level():
    return sum(
        region_type(x, y).value
        for x, y in product(range(x_target + 1), range(y_target + 1))
    )


def a_star(start, goal):
    """
    A* search from state `start` to `goal`.

    state is x, y, tool.
    """
    frontier = []
    heappush(frontier, (0, start))
    cost_so_far = defaultdict(lambda: inf)
    came_from = {start: None}
    cost_so_far[start] = 0

    while frontier:
        current = heappop(frontier)[1]
        if heuristic(current, goal) == 0:
            break
        for next in moves(current):
            new_cost = cost_so_far[current] + cost(current, next)
            if new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost + heuristic(next, goal)
                heappush(frontier, (priority, next))
                came_from[next] = current

    return came_from, cost_so_far, current


@cache
def heuristic(current, goal) -> float:
    "Minimum estimation of cost to get from a to goal. Also used to determine when we have reached goal (heuristic returns 0)."
    (x1, y1), tool1 = current
    (x2, y2), tool2 = goal
    return abs(x1 - x2) + abs(y1 - y2) + 7 * int(tool1 != tool2)


def cost(current, next):
    """Switching tools takes 7 minutes, moving takes 1"""
    (x1, y1), current_tool = current
    (x2, y2), next_tool = next
    if current_tool != next_tool:
        assert x1 == x2 and y1 == y2
        return 7
    return 1


def print_path(came_from, start, goal):
    step = goal
    while step != start:
        (x, y), _ = step
        print(step, region_type(x, y))
        step = came_from[step]
    (x, y), _ = step
    print(step, region_type(x, y))


@cache
def moves(current):
    (x, y), tool = current
    candidates = {
        ((x + 1, y), tool),
        ((x, y + 1), tool),
        ((x - 1, y), tool),
        ((x, y - 1), tool),
    }
    candidates.update(((x, y), new_tool) for new_tool in set(Tool) - {tool})
    return (pos for pos in candidates if valid_position(pos))


@cache
def valid_position(pos):
    (x, y), tool = pos
    if x < 0 or y < 0:
        return False
    match region_type(x, y):
        case Region.ROCKY:
            return tool != Tool.NEITHER
        case Region.WET:
            return tool != Tool.TORCH
        case Region.NARROW:
            return tool != Tool.CLIMBING_GEAR


depth, x_target, y_target = 7740, 12, 763
# depth, x_target, y_target = 510, 10, 10

print(f"Part 1: {risk_level()}")  # 9899, ~1s

start = ((0, 0), Tool.TORCH)
goal = ((x_target, y_target), Tool.TORCH)
_, cost_so_far, current = a_star(start, goal)
print(f"Part 2: {cost_so_far[goal]}")  # 1051, ~4s

Part 1: 9899
Part 2: 1051


In [69]:
# Part 1 as dynamic programming
def erosion(x, y):
    if (x, y) in erosion_value:
        return erosion_value[(x, y)]
    if x == 0 or y == 0:
        geo = 16807 * x + 48271 * y
    elif x == x_target and y == y_target:
        geo = 0
    else:
        geo = erosion(x - 1, y) * erosion(x, y - 1)
    erosion_value[(x, y)] = (geo + depth) % 20183
    return erosion_value[(x, y)]


erosion_value = {}

# sum(erosion(x, y) % 3 for (x, y) in product(range(x_target + 1), range(y_target + 1)))

In [105]:
# Day 23: Experimental Emergency Teleportation
def manhattan_distance(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1]) + abs(a[2] - b[2])


def in_range(a, b):
    "Determine if bot a is within range of bot b:s radius"
    return manhattan_distance(a, b) <= b[3]


lines = open("2018/23.txt").read().splitlines()
bots = {
    tuple(int(x) for x in re.findall("-?\d+", line)) for line in lines
}  # x, y, z, r
strongest = max(bots, key=lambda bot: bot[3])

print(f"Part 1: {sum(1 for bot in bots if in_range(bot, strongest))}")  # 580

Part 1: 580


In [100]:
lines = open("2018/23.txt").read().splitlines()
bots = {
    tuple(int(x) for x in re.findall("-?\d+", line)) for line in lines
}  # x, y, z, r

In [101]:
bots

{(27520883, 94424755, 67085913, 91215536),
 (-37926485, 61922522, 17510771, 64199966),
 (76486463, 61831912, 39116692, 79618794),
 (24364404, 94239329, 59905645, 80693211),
 (-55738512, 177576133, 37169545, 67770177),
 (15509423, 64915706, 37008353, 80922930),
 (78113323, 66317558, 31291739, 77906273),
 (59725290, 69406960, 32496366, 63812280),
 (79494848, 85091404, 148243006, 57010623),
 (65533832, 71131969, 38308556, 77158243),
 (-14015851, 76375216, 44783792, 65037679),
 (73926194, 53657964, 28934722, 67341315),
 (-19742397, 40002187, 72995758, 98553229),
 (50763298, 55281743, 48315648, 61935508),
 (-160835025, 79462165, 30046679, 70503208),
 (26481981, 94442073, 43161720, 66269441),
 (62085157, 50013307, 51444173, 81654357),
 (20632191, 51695767, -24237412, 63311648),
 (80161539, 59530252, 29659877, 71535334),
 (-134952467, 19332844, 49245959, 60892576),
 (-19530639, 40152556, 62018546, 87214112),
 (36793040, 79450374, 33470007, 51897266),
 (28431692, 90143704, 30577298, 51336404),